## Reference: 

[Liwen Huang IBM DS Capstone](https://github.com/alisonhuang1988/Coursera-capstone)

In [1]:
# pip install geopy

In [2]:
# pip install --upgrade pip

In [3]:
# pip install geocoder

In [4]:
# pip install bs4

In [5]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


In [6]:
url="https://en.wikipedia.org/wiki/List_of_neighborhoods_in_Orlando,_Florida"
tables = pd.read_html(url)
df = tables[0]

In [7]:
df.drop(columns=['Annexed[2][3]','Notes'], inplace=True)
df.head()

,Name
0,33rd Street Industrial
1,Airport North
2,Audubon Park
3,Azalea Park
4,Bal Bay


In [8]:
df.shape

(125, 1)

In [9]:
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Orlando, FL'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [10]:
coords = [ get_latlng(neighborhood) for neighborhood in df["Name"].tolist() ]
coords

[[28.505090000000052, -81.43109999999996],
 [28.46302000000003, -81.30760999999995],
 [28.565170000000023, -81.34261999999995],
 [28.532910000000072, -81.31040999999999],
 [28.42758235692473, -81.5695418283814],
 [28.566400000000044, -81.32508999999999],
 [28.521755050225536, -81.32326527843759],
 [28.56951898485235, -81.30354428488796],
 [28.436500000000024, -81.34972999999997],
 [28.508170000000064, -81.31946999999997],
 [28.545970000000068, -81.38908999999995],
 [28.522924052468746, -81.29789956304589],
 [28.526190000000042, -81.44660999999996],
 [28.514050000000054, -81.41722999999996],
 [28.54838061813682, -81.4127999726369],
 [28.537570000000073, -81.39774999999997],
 [28.52905000000004, -81.41218999999995],
 [28.571090000000027, -81.39372999999995],
 [28.557960000000037, -81.34398999999996],
 [28.55905000000007, -81.35924999999997],
 [28.549500000000023, -81.35764999999998],
 [28.517370000000028, -81.33510999999999],
 [28.555820029487748, -81.40924580768983],
 [28.55563000000006

In [11]:
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [12]:
df['Latitude'] = df_coords['Latitude']
df['Longitude'] = df_coords['Longitude']
df.head()

,Name,Latitude,Longitude
0,33rd Street Industrial,28.505090,-81.431100
1,Airport North,28.463020,-81.307610
2,Audubon Park,28.565170,-81.342620
3,Azalea Park,28.532910,-81.310410
4,Bal Bay,28.427582,-81.569542


In [13]:
# get the coordinates of Orlando
address = 'Orlando, FL'

geolocator = Nominatim(user_agent="http")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Orlando, FL {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Orlando, FL 28.5421109, -81.3790304.


In [14]:
# create map of Orlando using latitude and longitude values
map_ol = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(df['Latitude'], df['Longitude'], df['Name']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_ol)  
    
map_ol

## Define Foursquare Credentials and Version

**Make sure that you have created a Foursquare developer account and have your credential handy.**

**To obtain access token follow these steps.**

1.Go to **www.developer.foursquare.com**.

2.Click **Create Account**.

3.Sign up for Foursquare. 

4.Log in to Foursquare.

5.Click **"My Apps"** --> Click **"Create New App"**.

6.APP NAME: mads-project; Application URL: https://www.madsmilestoneproject.com

7.Set the "Redirect URL" under "Web Addresses" to https://www.google.com 

8.Paste and enter the following url in your web browser (replace YOUR_CLIENT_ID with your actual client id): https://foursquare.com/oauth2/authenticate?client_id=YOUR_CLIENT_ID&response_type=code&redirect_uri=https://www.google.com  
This should redirect you to a google page requesting permission to make the connection.

9.Accept and then look at the url of your web browser (take note at the CODE part of the url to use in step 5)

It should look like https://www.google.com/?code=CODE

10.Copy the code value from the previous step.

Paste and enter the following into your web browser **(replace placeholders with actual values)**:  https://foursquare.com/oauth2/access_token?client_id=YOUR_CLIENT_ID&client_secret=YOUR_CLIENT_SECRET&grant_type=authorization_code&redirect_uri=https://www.google.com&code=CODE.

11.When you paste the link , This should lead you to a page that gives you your access token.

In [15]:
CLIENT_ID = 'TK1YATXVPZUAE2E3TBVLRNEU2CK2SPZTBG4SP3REWVQSA3FX' 
CLIENT_SECRET = 'WCESSVH22IDI0MU24LXD0CI330RJRWGWXRFTAJIL5K0XCAOQ' # your Foursquare Secret
ACCESS_TOKEN = 'RD300UWHGIXDZ0GFIOKUBETXCDDOLVXN0G1HMXA0GUCSSEN0' 
VERSION = '20180604'
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: TK1YATXVPZUAE2E3TBVLRNEU2CK2SPZTBG4SP3REWVQSA3FX
CLIENT_SECRET:WCESSVH22IDI0MU24LXD0CI330RJRWGWXRFTAJIL5K0XCAOQ


In [16]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Street', 
                  'Street Latitude', 
                  'Street Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [17]:
# Run the above function on each location and create a new dataframe called location_venues and display it.
location_venues = getNearbyVenues(names=df['Name'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

33rd Street Industrial
Airport North
Audubon Park
Azalea Park
Bal Bay
Baldwin Park
Bel Air
Beltway Commerce Center
Boggy Creek
Bryn Mawr
Callahan
Camellia Gardens
Carver Shores
Catalina
Central Business District
City of Orlando/GOAA
Clear Lake
College Park
Colonial Town Center
Colonialtown North
Colonialtown South
Conway
Countryside
Coytown
Crescent Park
Delaney Park
Dixie Belle
Dover Estates
Dover Manor
Dover Shores East
Dover Shores West
Eagles Nest
East Central Park
East Park
Education Village
Engelwood Park
Florida Center
Florida Center North
Haralson Estates
Hibiscus
Holden Heights
Holden/Parramore
Johnson Village
Kirkman North
Kirkman South
Lake Cherokee
Lake Como
Lake Copeland
Lake Davis/Greenwood
Lake Dot
Lake Eola Heights
Lake Fairview
Lake Formosa
Lake Fredrica
Lake Holden
Lake Mann Estates
Lake Mann Gardens
Lake Nona Central
Lake Nona Estates
Lake Nona South
Lake Richmond
Lake Shore Village
Lake Sunset
Lake Terrace
Lake Underhill
Lake Weldona
Lancaster Park
LaVina
Lawsona/Fe

In [18]:

location_venues.head()

,Street,Street Latitude,Street Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,33rd Street Industrial,28.50509,-81.4311,International Food Club,28.508228,-81.432516,Farmers Market
1,33rd Street Industrial,28.50509,-81.4311,Hertz,28.505224,-81.427012,Rental Car Location
2,33rd Street Industrial,28.50509,-81.4311,Orlando Gun Club,28.502050,-81.429336,Athletics & Sports
3,33rd Street Industrial,28.50509,-81.4311,Toyota of Orlando - Collison Center,28.505346,-81.426936,Auto Workshop
4,33rd Street Industrial,28.50509,-81.4311,Jati Patio Furniture,28.509205,-81.430962,Furniture / Home Store


In [19]:

location_venues.groupby('Street').count()

,Street Latitude,Street Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Street,,,,,,
33rd Street Industrial,5,5,5,5,5,5
Airport North,63,63,63,63,63,63
Audubon Park,22,22,22,22,22,22
Azalea Park,8,8,8,8,8,8
Bal Bay,1,1,1,1,1,1
Baldwin Park,20,20,20,20,20,20
Bel Air,1,1,1,1,1,1
Beltway Commerce Center,4,4,4,4,4,4
Boggy Creek,2,2,2,2,2,2


In [20]:

# get the List of Unique Categories
print('There are {} uniques categories.'.format(len(location_venues['Venue Category'].unique())))

There are 255 uniques categories.


In [21]:

location_venues.shape

(1214, 7)

In [22]:
# one hot encoding
venues_onehot = pd.get_dummies(location_venues[['Venue Category']], prefix="", prefix_sep="")

# add street column back to dataframe
venues_onehot['Street'] = location_venues['Street'] 

# move street column to the first column
fixed_columns = [venues_onehot.columns[-1]] + list(venues_onehot.columns[:-1])

#fixed_columns
venues_onehot = venues_onehot[fixed_columns]

venues_onehot.head()

,Street,ATM,Accessories Store,Adult Boutique,Airport,Airport Service,American Restaurant,Animal Shelter,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bistro,Board Shop,Boat Rental,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Burger Joint,Bus Line,Bus Station,Bus Stop,Business Service,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cemetery,Check Cashing Service,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cuban Restaurant,Cupcake Shop,Currency Exchange,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Dry Cleaner,Electronics Store,Event Service,Event Space,Eye Doctor,Farm,Farmers Market,Fast Food Restaurant,Financial or Legal Service,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Health & Beauty Service,History Museum,Hobby Shop,Home Service,Hookah Bar,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Ice Cream Shop,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Mattress Store,Mediterranean Restaurant,Memorial Site,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Mobile Phone Shop,Motel,Movie Theater,Moving Target,Music Venue,Nature Preserve,New American Restaurant,Nightclub,North Indian Restaurant,Office,Optical Shop,Outdoor Sculpture,Paper / Office Supplies Store,Park,Pawn Shop,Performing Arts Venue,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Pier,Pizza Place,Playground,Poke Place,Pool,Print Shop,Professional & Other Places,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Resort,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Sausage Shop,Science Museum,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,South American Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Street Food Gathering,Supermarket,Supplement Shop,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tanning Salon,Tapas Restaurant,Tattoo Parlor,Taxi Stand,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vintage Store,Tiki Bar,Toll Booth,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Veterans' Organization,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Waterfront,Weight Loss Center,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,33rd Street Industrial,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [23]:
orlando_grouped = venues_onehot.groupby('Street').mean().reset_index()
orlando_grouped.head()

,Street,ATM,Accessories Store,Adult Boutique,Airport,Airport Service,American Restaurant,Animal Shelter,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bistro,Board Shop,Boat Rental,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Burger Joint,Bus Line,Bus Station,Bus Stop,Business Service,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cemetery,Check Cashing Service,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cuban Restaurant,Cupcake Shop,Currency Exchange,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Dry Cleaner,Electronics Store,Event Service,Event Space,Eye Doctor,Farm,Farmers Market,Fast Food Restaurant,Financial or Legal Service,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Health & Beauty Service,History Museum,Hobby Shop,Home Service,Hookah Bar,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Ice Cream Shop,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Mattress Store,Mediterranean Restaurant,Memorial Site,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Mobile Phone Shop,Motel,Movie Theater,Moving Target,Music Venue,Nature Preserve,New American Restaurant,Nightclub,North Indian Restaurant,Office,Optical Shop,Outdoor Sculpture,Paper / Office Supplies Store,Park,Pawn Shop,Performing Arts Venue,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Pier,Pizza Place,Playground,Poke Place,Pool,Print Shop,Professional & Other Places,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Resort,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Sausage Shop,Science Museum,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,South American Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Street Food Gathering,Supermarket,Supplement Shop,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tanning Salon,Tapas Restaurant,Tattoo Parlor,Taxi Stand,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vintage Store,Tiki Bar,Toll Booth,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Veterans' Organization,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Waterfront,Weight Loss Center,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,33rd Street Industrial,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.2,0.0,0.2,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000

In [24]:
orlando_grouped.shape

(120, 256)

In [25]:
# What are the top 5 venues/facilities nearby profitable real estate investments?#

num_top_venues = 5

for hood in orlando_grouped['Street']:
    print("----"+hood+"----")
    temp = orlando_grouped[orlando_grouped['Street'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----33rd Street Industrial----
                    venue  freq
0     Rental Car Location   0.2
1      Athletics & Sports   0.2
2  Furniture / Home Store   0.2
3          Farmers Market   0.2
4           Auto Workshop   0.2


----Airport North----
                  venue  freq
0                 Hotel  0.19
1        Breakfast Spot  0.08
2  Gym / Fitness Center  0.06
3   American Restaurant  0.06
4           Pizza Place  0.05


----Audubon Park----
               venue  freq
0        Coffee Shop  0.09
1        Karaoke Bar  0.09
2  Convenience Store  0.05
3             Market  0.05
4        Record Shop  0.05


----Azalea Park----
                       venue  freq
0             Breakfast Spot  0.25
1          Electronics Store  0.12
2              Grocery Store  0.12
3  Latin American Restaurant  0.12
4             Discount Store  0.12


----Bal Bay----
             venue  freq
0              Gym   1.0
1              ATM   0.0
2      Pet Service   0.0
3      Music Venue   0.0
4  Nature Pre

In [26]:
# Define a function to return the most common venues/facilities nearby real estate investments#

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [27]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Street']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

In [28]:
# create a new dataframe
venues_sorted = pd.DataFrame(columns=columns)
venues_sorted['Street'] = orlando_grouped['Street']

for ind in np.arange(orlando_grouped.shape[0]):
    venues_sorted.iloc[ind, 1:] = return_most_common_venues(orlando_grouped.iloc[ind, :], num_top_venues)

venues_sorted.head()

,Street,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,33rd Street Industrial,Rental Car Location,Athletics & Sports,Furniture / Home Store,Farmers Market,Auto Workshop,ATM,Paper / Office Supplies Store,Performing Arts Venue,Pawn Shop,Park
1,Airport North,Hotel,Breakfast Spot,Gym / Fitness Center,American Restaurant,Pizza Place,Hotel Pool,Clothing Store,Mexican Restaurant,Bistro,Restaurant
2,Audubon Park,Coffee Shop,Karaoke Bar,Convenience Store,Market,Record Shop,Diner,Ramen Restaurant,Shopping Plaza,Café,Food & Drink Shop
3,Azalea Park,Breakfast Spot,Electronics Store,Grocery Store,Latin American Restaurant,Discount Store,Gas Station,Convenience Store,Outdoor Sculpture,Paper / Office Supplies Store,Optical Shop
4,Bal Bay,Gym,ATM,Pet Service,Music Venue,Nature Preserve,New American Restaurant,Nightclub,North Indian Restaurant,Office,Optical Shop


In [29]:
venues_sorted.shape

(120, 11)

In [30]:
orlando_grouped.shape

(120, 256)

In [31]:
orlando_grouped = df

In [32]:
#Distribute in 3 Clusters

# set number of clusters
kclusters = 3

orlando_grouped_clustering = orlando_grouped.drop('Name', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(orlando_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:50]

C:\Users\aliso\AppData\Local\Temp/ipykernel_32820/1514839868.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  orlando_grouped_clustering = orlando_grouped.drop('Name', 1)


array([0, 2, 1, 1, 0, 1, 1, 1, 2, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1,
       0, 1, 2, 1, 1, 1, 1, 1, 1, 0, 1, 2, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 1])

In [33]:
orlando_grouped_clustering = df
orlando_grouped_clustering.head()

,Name,Latitude,Longitude
0,33rd Street Industrial,28.505090,-81.431100
1,Airport North,28.463020,-81.307610
2,Audubon Park,28.565170,-81.342620
3,Azalea Park,28.532910,-81.310410
4,Bal Bay,28.427582,-81.569542


In [34]:
orlando_grouped_clustering.shape

(125, 3)

In [35]:
df.shape

(125, 3)

In [36]:
# add clustering labels
orlando_grouped_clustering['Cluster Labels'] = kmeans.labels_

# merge london_grouped with london_data to add latitude/longitude for each neighborhood
orlando_grouped_clustering = orlando_grouped_clustering.join(venues_sorted.set_index('Street'), on='Name')

orlando_grouped_clustering.head(30) # check the last columns!

,Name,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,33rd Street Industrial,28.505090,-81.431100,0,Rental Car Location,Athletics & Sports,Furniture / Home Store,Farmers Market,Auto Workshop,ATM,Paper / Office Supplies Store,Performing Arts Venue,Pawn Shop,Park
1,Airport North,28.463020,-81.307610,2,Hotel,Breakfast Spot,Gym / Fitness Center,American Restaurant,Pizza Place,Hotel Pool,Clothing Store,Mexican Restaurant,Bistro,Restaurant
2,Audubon Park,28.565170,-81.342620,1,Coffee Shop,Karaoke Bar,Convenience Store,Market,Record Shop,Diner,Ramen Restaurant,Shopping Plaza,Café,Food & Drink Shop
3,Azalea Park,28.532910,-81.310410,1,Breakfast Spot,Electronics Store,Grocery Store,Latin American Restaurant,Discount Store,Gas Station,Convenience Store,Outdoor Sculpture,Paper / Office Supplies Store,Optical Shop
4,Bal Bay,28.427582,-81.569542,0,Gym,ATM,Pet Service,Music Venue,Nature Preserve,New American Restaurant,Nightclub,North Indian Restaurant,Office,Optical Shop
5,Baldwin Park,28.566400,-81.325090,1,Sushi Restaurant,Burger Joint,Candy Store,Gift Shop,Mexican Restaurant,Grocery Store,Liquor Store,Pharmacy,Asian Restaurant,Smoke Shop
6,Bel Air,28.521755,-81.323265,1,Gym,ATM,Pet Service,Music Venue,Nature Preserve,New American Restaurant,Nightclub,North Indian Restaurant,Office,Optical Shop
7,Beltway Commerce Center,28.569519,-81.303544,1,Insurance Office,Café,American Restaurant,Farm,ATM,Paper / Office Supplies Store,Peruvian Restaurant,Performing Arts Venue,Pawn Shop,Park
8,Boggy Creek,28.436500,-81.349730,2,Dog Run,Intersection,ATM,Outdoor Sculpture,Peruvian Restaurant,Performing Arts Venue,Pawn Shop,Park,Paper / Office Supplies Store,Optical Shop
9,Bryn Mawr,28.508170,-81.319470,1,Cocktail Bar,Lounge,ATM,Pet Service,Nature Preserve,New American Restaurant,Nightclub,North Indian Restaurant,Office,Optical Shop


In [37]:
# Create Map

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(orlando_grouped_clustering['Latitude'], orlando_grouped_clustering['Longitude'], orlando_grouped_clustering['Name'], orlando_grouped_clustering['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-3],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [38]:

orlando_grouped_clustering.loc[orlando_grouped_clustering['Cluster Labels'] == 0, orlando_grouped_clustering.columns[[1] + list(range(5, orlando_grouped_clustering.shape[1]))]].head()

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,28.505090,Athletics & Sports,Furniture / Home Store,Farmers Market,Auto Workshop,ATM,Paper / Office Supplies Store,Performing Arts Venue,Pawn Shop,Park
4,28.427582,ATM,Pet Service,Music Venue,Nature Preserve,New American Restaurant,Nightclub,North Indian Restaurant,Office,Optical Shop
12,28.526190,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,28.514050,Hotel,Convenience Store,ATM,Outdoor Sculpture,Peruvian Restaurant,Performing Arts Venue,Pawn Shop,Park,Paper / Office Supplies Store
14,28.548381,Gas Station,Convenience Store,Coffee Shop,Office,Deli / Bodega,Paper / Office Supplies Store,Peruvian Restaurant,Performing Arts Venue,Pawn Shop


In [39]:

orlando_grouped_clustering.loc[orlando_grouped_clustering['Cluster Labels'] == 1, orlando_grouped_clustering.columns[[1] + list(range(5, orlando_grouped_clustering.shape[1]))]].head()

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,28.565170,Karaoke Bar,Convenience Store,Market,Record Shop,Diner,Ramen Restaurant,Shopping Plaza,Café,Food & Drink Shop
3,28.532910,Electronics Store,Grocery Store,Latin American Restaurant,Discount Store,Gas Station,Convenience Store,Outdoor Sculpture,Paper / Office Supplies Store,Optical Shop
5,28.566400,Burger Joint,Candy Store,Gift Shop,Mexican Restaurant,Grocery Store,Liquor Store,Pharmacy,Asian Restaurant,Smoke Shop
6,28.521755,ATM,Pet Service,Music Venue,Nature Preserve,New American Restaurant,Nightclub,North Indian Restaurant,Office,Optical Shop
7,28.569519,Café,American Restaurant,Farm,ATM,Paper / Office Supplies Store,Peruvian Restaurant,Performing Arts Venue,Pawn Shop,Park


In [40]:
orlando_grouped_clustering.loc[orlando_grouped_clustering['Cluster Labels'] == 2, orlando_grouped_clustering.columns[[1] + list(range(5, orlando_grouped_clustering.shape[1]))]].head()

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,28.463020,Breakfast Spot,Gym / Fitness Center,American Restaurant,Pizza Place,Hotel Pool,Clothing Store,Mexican Restaurant,Bistro,Restaurant
8,28.436500,Intersection,ATM,Outdoor Sculpture,Peruvian Restaurant,Performing Arts Venue,Pawn Shop,Park,Paper / Office Supplies Store,Optical Shop
24,28.460190,ATM,Pet Service,Music Venue,Nature Preserve,New American Restaurant,Nightclub,North Indian Restaurant,Office,Optical Shop
33,28.422470,Playground,Park,ATM,Peruvian Restaurant,Music Venue,Nature Preserve,New American Restaurant,Nightclub,North Indian Restaurant
57,28.398653,Movie Theater,Moving Target,Music Venue,Nature Preserve,New American Restaurant,Nightclub,North Indian Restaurant,Office,Optical Shop


# Results and Discussion Section

We may analyze our results according to the three clusters we have produced. Even though, all clusters could praise an optimal range of facilities and amaentities, we have found two main patterns. The first pattern we are referring to, i.e. Cluster 0, have more Park and Gym, the second pattern we are referring to, i.e. Cluster 1 and 2, have more restaurants and Lounge.